In [1]:
from tools import *
import sys

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils

In [3]:
def test(ecofile,infile,outfile,ite=1):
    # ecofile, ecoli model file prefix
    # infile, the model file for gap-filling
    # outfile, the output model file
    eco = load_ecoli_model(ecofile+'.mat',ecofile+'.xml')
    model = pickle.load(open(infile,'rb'))
    
    print('Convert eco to universal model') 
    universal = convert_eco_to_metacyc_kegg(eco,model)
    return universal,model

In [4]:
def print_rxn_of_met(model,met_id):
    met = model.metabolites.get_by_id(met_id)
    for rxn in met.reactions:
        if 'biomass' in rxn.id.lower():continue
        if rxn.metabolites[met]<0: continue
        print(rxn.id,rxn.reaction)

universal,model = test(ecofile_prefix,model_file3,outfile1)

utils.test_met_production(universal,'CPD-15815_c')

print_rxn_of_met(model,'CPD-15815_c')

In [5]:
def do_block(model,rxn_id):
    rxn = model.reactions.get_by_id(rxn_id) # differnt number of protons in metacyc
    model.remove_reactions([rxn])

In [6]:
def some_manual_curations(model):
    try:
        # based on metacyc website
        rxn = model.reactions.get_by_id('NADH-DEHYDROG-A-RXN')
        rxn.lower_bound = 0
        print(rxn.reaction)
    except: pass
    
    # enable only secretion of cit
    rxn = model.reactions.get_by_id('Exchange_CIT')
    rxn.upper_bound = 0
    print(rxn.reaction)
    
    do_block(model,'GLUt2rpp') # differnt number of protons in metacyc
    do_block(model,'RXN-11039')# break the loop


In [7]:
def block_some_universal_rxns(model):
    do_block(model,'GLUt2rpp') # differnt number of protons in metacyc
    do_block(model,'PROt2rpp') # differnt number of protons in metacyc


In [8]:
def add_missing_tag(rxn):
    rxn_clean = rxn.copy()
    # add _missing tag for those genes
    new_gr = rxn_clean.gene_reaction_rule
    for gene in parse_gr(rxn_clean.gene_reaction_rule):
        new_gr = new_gr.replace(gene,gene+'_missing')
    rxn_clean.gene_reaction_rule = new_gr
    return rxn_clean

In [9]:
def manually_fill_ions_uptake(universal,model):
    rxns_to_add = []
    def _add_rxn(rxn_id):
        rxn = universal.reactions.get_by_id(rxn_id)
        rxn_clean = add_missing_tag(rxn)
        
        rxns_to_add.append(rxn_clean)
        print(rxn_clean.id, rxn_clean.reaction,rxn_clean.gene_reaction_rule)
        print()
    
    # K+ 
    _add_rxn('TRANS-RXN-3')
    
    # CO+2_c
    _add_rxn('TRANS-RXN-141A_copy2')
    
    # CU+2_c
    _add_rxn('RXN0-14_copy2')
    
    # FE+2_c 
    _add_rxn('TRANS-RXN-8_copy2')
    
    # FE+3_c 
    _add_rxn('3.6.3.30-RXN')
    
    # MN+2_c
    _add_rxn('RXN0-16_copy2')
    
    model.add_reactions(rxns_to_add)

In [10]:
def gap_filling_for_lipid(universal,model):
    for met_id in ['CPD-12819_p','CPD-17086_p','KDO2-LIPID-IVA_e']:
        s = utils.test_met_production(model,met_id)
        rxns_to_add = []
        if s.objective_value == 0:
            su = utils.test_met_production(universal,met_id)
            for rxn_id in su.fluxes.index:
                if rxn_id == 'tmp_sink': continue
                if rxn_id == 'ATPASE-RXN_copy1': continue
                if rxn_id.startswith('EX_'):continue
                if su.fluxes[rxn_id] == 0: continue
                try: model.reactions.get_by_id(rxn_id)
                except: 
                    rxn = universal.reactions.get_by_id(rxn_id)
                    rxn = add_missing_tag(rxn)
                    rxns_to_add.append(rxn)
                    print('  ',rxn.id,rxn.reaction)
        if len(rxns_to_add) >0: model.add_reactions(rxns_to_add)
        print(len(rxns_to_add), 'for lipid',met_id)
        model.repair()
       

In [11]:
def run_gapfilling(ecofile,infile,outfile,ite=1):
    # ecofile, ecoli model file prefix
    # infile, the model file for gap-filling
    # outfile, the output model file
    eco = load_ecoli_model(ecofile+'.mat',ecofile+'.xml')
    block_some_universal_rxns(eco)
    
    model = pickle.load(open(infile,'rb'))
    
    some_manual_curations(model)
    
    print('Convert eco to universal model') 
    universal = convert_eco_to_metacyc_kegg(eco,model)
    
    print()
    print('Test if there is consistant prediction before and after conversion')
    report_model_status(eco)
    report_model_status(universal)
    
    universal.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
    eco.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
    print(universal.optimize())
    print(eco.optimize())
    print()
    
    
    print('Test if model can produce components in biomass with the medium without YE')
    utils.set_yeast_extraction(model,lb=0,ub=0)
    check_if_model_can_produce_mets_in_biomass_one_by_one(model)
    print()
    
    print('Do gap-filling one by one')
    manually_fill_ions_uptake(universal,model)
    #gap_filling_for_lipid(universal,model)
    for i in range(ite):
        for met in  model.reactions.Biomass_v1.reactants:
            if met.id == 'ions_c': continue
            #if met.id == 'lipid_c': continue
            do_gap_filling(met,model,universal)
        
    print('Test if model can produce components in biomass with the medium without YE, 1 glucose')
    model.reactions.Exchange_Glucopyranose.lower_bound = 0
    model.reactions.Exchange_Glucopyranose.upper_bound = 1
    check_if_model_can_produce_mets_in_biomass_one_by_one(model)
    print()
    
    print('Test biomass production with/wihout YE')
    print(utils.test_biomass_production(model))
    print()
    
    print('Update missing genes')
    model = update_missing_genes(model,1e-3,report=True,coverage=45)
    print()
    
    print('Save model')
    save_pickle_model(model,outfile)

In [12]:
ecofile_prefix ='../../../ComplementaryData/iML1515'
model_file1 = '../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass.pkl'
outfile1    = '../Results/halo_metacycmodel_100_curated_compart_with_eco_medium_biomass_gapfilled.pkl'

model_file2 = '../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass.pkl'
outfile2    = '../Results/halo_metacyc_kegg_curated_compart_with_eco_medium_biomass_gapfilled.pkl'

model_file3 = '../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass.pkl'
outfile3    = '../Results/halo_metacycmodel_100_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl'

model_file4 = '../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass.pkl'
outfile4    = '../Results/halo_metacyc_kegg_curated_compart_with_eco_without_missing_medium_biomass_gapfilled.pkl'

In [13]:
run_gapfilling(ecofile_prefix,model_file1,outfile1)

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpmq7cqvpg.lp
Reading time = 0.02 seconds
: 2399 rows, 4618 columns, 21886 nonzeros


cobra/core/group.py:109 UserWarning: need to pass in a list


NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x1329c6e5c0>
<Solution 0.877 at 0x1329c6e5c0>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0
   rna_c rna 0.0
   ions_c ions 0.0
   lipids_c lipids 0.0
   10-FORMYL-THF_c 10-Formyltetrahydrofolate 0.0
   2fe2s_c [2Fe-2S] iron-sulfur cluster 0.0
 

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpbyxfeu6s.lp
Reading time = 0.02 seconds
: 2399 rows, 4626 columns, 21902 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpatoduipv.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
After gap-filling: 45.81588957869483
A

...

After gap-filling: 28.59411504424787
Added reactions:
   PANTOTHENATE-KIN-RXN ATP_c + PANTOTHENATE_c --> 4-P-PANTOTHENATE_c + ADP_c + PROTON_c



FAD_c
Before gap-filling: 24.666111766092882
No need for gap-filling



CPD-15815_c
Before gap-filling: 1000.0
No need for gap-filling



METHYLENE-THF_c
Before gap-filling: 31.85431567907709
No need for gap-filling



CPD0-2278_p
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpe6vf2ocp.lp
Reading time = 0.02 seconds
: 2400 rows, 4668 columns, 22058 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp4iq8xt6b.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'TETHYDPICSUCC-RXN' since it already exists.
Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.


After gap-filling: 15.87573512906846
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpov0pgely.lp
Reading time = 0.08 seconds
: 2400 rows, 4670 columns, 22066 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmprhkwl_19.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.


After gap-filling: 34.08858340318537
Added reactions:
   NAD-SYNTH-NH3-RXN AMMONIA_c + ATP_c + DEAMIDO-NAD_c --> AMP_c + NAD_c + PPI_c + PROTON_c



NADP_c
Before gap-filling: 32.33367521367518
No need for gap-filling



PROTOHEME_c
Before gap-filling: 22.745700410907727
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 112.06515699333967
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 46.878323997311234
No need for gap-filling



SIROHEME_c
Before gap-filling: 20.7887470126323
No need for gap-filling



SUC-COA_c
Before gap-filling: 26.814398340248985
No need for gap-filling



THF_c
Before gap-filling: 33.25824254037781
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 39.95221020092737
No need for gap-filling



CPD-9649_c
Before gap-filling: 19.174930555555566
No need for gap-filling



ATP_c
Before gap-filling: 51.89763348996159
No need for gap-filling



Test if model can produce components in biomass with the medium without YE

In [14]:
run_gapfilling(ecofile_prefix,model_file2,outfile2)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmppro81a49.lp
Reading time = 0.03 seconds
: 3310 rows, 6314 columns, 29474 nonzeros


cobra/core/group.py:109 UserWarning: need to pass in a list


NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x131fd98da0>
<Solution 0.877 at 0x131fd98f60>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 965.5793048215227
   rna_c rna 169.7761944719367
   ions_c ions 0.0
   lipids_c lipids 0.0
   10-FORMYL-THF_c 10-Formyltetrahydrofolate 0.0
   2fe2s_c [2Fe-2

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp7pnb4s44.lp
Reading time = 0.03 seconds
: 3310 rows, 6322 columns, 29490 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmplfj809tp.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
After gap-filling: 57.97392327133858
A

...

After gap-filling: 43.72519195612425
Added reactions:
   PANTOTHENATE-KIN-RXN ATP_c + PANTOTHENATE_c --> 4-P-PANTOTHENATE_c + ADP_c + PROTON_c



FAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp058u7ypn.lp
Reading time = 0.03 seconds
: 3311 rows, 6346 columns, 29572 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp5yo41nvo.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 80.25346080305911
Added reactions:
   DM_CARBOXYL-GROUP_c CARBOXYL-GROUP_c --> 



CPD-15815_c
Before gap-filling: 1000.0
No need for gap-filling



METHYLENE-THF_c
Before gap-filling: 115.30923076923057
No need for gap-filling



CPD0-2278_p
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmptzsmm67o.lp
Reading time = 0.03 seconds
: 3311 rows, 6348 columns, 29574 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp7re01h48.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 54.05405405405408
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 109.87581151832472
No need for gap-filling



NADP_c
Before gap-filling: 103.12668304668284
No need for gap-filling



PROTOHEME_c
Before gap-filling: 62.50000000000001
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 333.33333333333354
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 156.41947826086937
No need for gap-filling



SIROHEME_c
Before gap-filling: 38.46153846153838
No need for gap-filling



SUC-COA_c
Before gap-filling: 90.90909090909082
No need for gap-filling



THF_c
Before gap-filling: 121.54216216216187
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 108.35023508137427
No need for gap-filling



CPD-9649_c
Before gap-filling: 35.714285714285715
No need for gap-filling



ATP_c
Before gap-filling: 162.2650515463921
No need for gap-filling



Test if model can produce

In [15]:
run_gapfilling(ecofile_prefix,model_file3,outfile3,ite=2)

cobra/core/group.py:109 UserWarning: need to pass in a list


Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp7yy2l29a.lp
Reading time = 0.02 seconds
: 2337 rows, 4472 columns, 21114 nonzeros
NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x1334adf438>
<Solution 0.877 at 0x1334adf438>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dn

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpg1n8em8t.lp
Reading time = 0.02 seconds
: 2337 rows, 4480 columns, 21130 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp5db_cgvs.lp
Reading time = 0.02 seconds
: 1872 row

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.


...

After gap-filling: 13.867548556430444
Added reactions:
   DM_D-ALANINE_p D-ALANINE_p --> 



NAD_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpycn_23ce.lp
Reading time = 0.02 seconds
: 2338 rows, 4530 columns, 21322 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpxgqkpj4k.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.
Ignoring reaction 'TRANS-RXN-114' since it already exists.


After gap-filling: 29.98601589103282
Added reactions:
   NAD-SYNTH-NH3-RXN AMMONIA_c + ATP_c + DEAMIDO-NAD_c --> AMP_c + NAD_c + PPI_c + PROTON_c



NADP_c
Before gap-filling: 28.637051490514892
No need for gap-filling



PROTOHEME_c
Before gap-filling: 20.514319294809017
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 104.97431578947368
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 39.65130206378931
No need for gap-filling



SIROHEME_c
Before gap-filling: 18.112910524511495
No need for gap-filling



SUC-COA_c
Before gap-filling: 24.438186864014845
No need for gap-filling



THF_c
Before gap-filling: 28.088973950026993
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmprkkl8sui.lp
Reading time = 0.02 seconds
: 2338 rows, 4532 columns, 21336 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpg9h22e4t.lp


Ignoring reaction 'SULFITE-REDUCT-RXN' since it already exists.
Ignoring reaction 'ADENYLYLSULFKIN-RXN' since it already exists.
Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'DM_CARBON-DIOXIDE_c' since it already exists.
Ignoring reaction 'TRANS-RXN-114' since it already exists.
Ignoring reaction 'DM_CARBOXYL-GROUP_c' since it already exists.


After gap-filling: 36.38798898071625
Added reactions:
   THZPSN3 ATP_c + CPD-12279_c + DEOXYXYLULOSE-5P_c + NADPH_c + PROTON_c + iscssh_c --> AMP_c + CARBON-DIOXIDE_c + 2.0 CPD-15815_c + NADP_c + PPI_c + THZ-P_c + iscs_c



CPD-9649_c
Before gap-filling: 17.120985093972774
No need for gap-filling



ATP_c
Before gap-filling: 46.20494971578489
No need for gap-filling



protein_c
Before gap-filling: 40.8807269094466
No need for gap-filling



dna_c
Before gap-filling: 501.2443601736902
No need for gap-filling



rna_c
Before gap-filling: 81.22851446636699
No need for gap-filling



lipids_c
Before gap-filling: 204.09698328983418
No need for gap-filling



10-FORMYL-THF_c
Before gap-filling: 27.220690365790826
No need for gap-filling



2fe2s_c
Before gap-filling: 75.6395652173913
No need for gap-filling



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 20.204726577437853
No need for gap-filling



CPD-7_c
Before gap-filling: 38.23538461538462
No need for gap-filling



S-ADENOSYLMET

In [16]:
run_gapfilling(ecofile_prefix,model_file4,outfile4,ite=2)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp_9cm86y0.lp
Reading time = 0.03 seconds
: 3252 rows, 6170 columns, 28714 nonzeros


cobra/core/group.py:109 UserWarning: need to pass in a list


NADH_c + PROTON_c + Ubiquinones_c --> NAD_c + Ubiquinols_c
 <-- CIT_e
Convert eco to universal model
Metacyc 16868
KEGG 37608
Metacyc 14768
KEGG 10278

Test if there is consistant prediction before and after conversion
Number of reactions: 2710
Number of metabolits: 1877
Number of compartments: 3 {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
Number of genes: 1516
Number of missing genes: 0
Number of reactions with missing genes: 0

Number of reactions: 2710
Number of metabolits: 1872
Number of compartments: 3 {'c': '', 'e': '', 'p': ''}
Number of genes: 1515
Number of missing genes: 0
Number of reactions with missing genes: 0

<Solution 0.877 at 0x132cae50b8>
<Solution 0.877 at 0x132cae50b8>

Test if model can produce components in biomass with the medium without YE
Biomass_v1
   protein_c protein 0.0
   dna_c dna 0.0
   rna_c rna 0.0
   ions_c ions 0.0
   lipids_c lipids 0.0
   10-FORMYL-THF_c 10-Formyltetrahydrofolate 0.0
   2fe2s_c [2Fe-2S] iron-sulfur cluster 0.0
 

Ignoring reaction 'TRANS-RXN-3' since it already exists.


   CPD-12819_p Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
   CPD-17086_p Phosphatidylethanolamine (dihexadec-9enoyl, n-C16:1) 0.0


Do gap-filling one by one
TRANS-RXN-3 K+_p + PROTON_p --> K+_c + PROTON_c ( b1363_missing and b1291_missing and b3290_missing ) or ( b1291_missing and b3849_missing and b3290_missing ) or b1250_missing or b3747_missing

TRANS-RXN-141A_copy2 CO+2_p --> CO+2_c b3040_missing or b3816_missing

RXN0-14_copy2 CU+2_p --> CU+2_c b3040_missing

TRANS-RXN-8_copy2 FE+2_p --> FE+2_c b3040_missing

3.6.3.30-RXN ATP_c + CPD-15815_c + FE+3_p --> ADP_c + CPD-16459_c + FE+3_c + PROTON_c 

RXN0-16_copy2 MN+2_p --> MN+2_c b3040_missing

protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmporr86wte.lp
Reading time = 0.03 seconds
: 3252 rows, 6178 columns, 28730 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmphgalu5h4.lp
Reading time = 0.02 seconds
: 1872 row

...

After gap-filling: 54.05405405405404
Added reactions:
   TRANS-RXN-62A D-ALANINE_p + PROTON_p --> D-ALANINE_c + PROTON_c



NAD_c
Before gap-filling: 110.34527607361917
No need for gap-filling



NADP_c
Before gap-filling: 104.26828985507274
No need for gap-filling



PROTOHEME_c
Before gap-filling: 62.49999999999999
No need for gap-filling



PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 333.33333333333337
No need for gap-filling



RIBOFLAVIN_c
Before gap-filling: 154.16811428571427
No need for gap-filling



SIROHEME_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpa8o_1j14.lp
Reading time = 0.02 seconds
: 3253 rows, 6194 columns, 28780 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpmb48586q.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros


Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.


After gap-filling: 71.42857142857143
Added reactions:
   DM_DIHYDROXYPENTANEDIONE_c DIHYDROXYPENTANEDIONE_c --> 



SUC-COA_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpzg20rb57.lp
Reading time = 0.03 seconds
: 3253 rows, 6196 columns, 28782 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpp6tq9dev.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for SUC-COA_c



THF_c
Before gap-filling: 122.35564625850338
No need for gap-filling



THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpvhh3odrv.lp
Reading time = 0.03 seconds
: 3253 rows, 6196 columns, 28782 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmponk75jza.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for THIAMINE-P

Ignoring reaction 'ATPASE-RXN_copy1' since it already exists.
Ignoring reaction 'TRANS-RXN-114' since it already exists.
Ignoring reaction 'DM_murein5p5p5p_p' since it already exists.


Gap-filling failed for lipids_c



10-FORMYL-THF_c
Before gap-filling: 118.07185995623695
No need for gap-filling



2fe2s_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpo6rk74kq.lp
Reading time = 0.02 seconds
: 3253 rows, 6196 columns, 28782 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpsewaegtr.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for 2fe2s_c



2-OCTAPRENYL-6-HYDROXYPHENOL_c
Before gap-filling: 43.478260869565226
No need for gap-filling



CPD-7_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmplgry9g6a.lp
Reading time = 0.03 seconds
: 3253 rows, 6196 columns, 28782 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp8mita73g.lp
Reading time = 0.02 seconds
: 1872 rows, 5420 columns, 21134 nonzeros
Gap-filling failed for CPD-7